In [5]:
import os
import nbformat
from nbformat.v4 import new_notebook, new_code_cell

def extract_cells_from_txt(file_path):
    """
    Extract code cells from a .txt file. 
    Content between 'EXECUTE' and 'STDOUT/STDERR' is extracted.
    """
    cells = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    inside_cell = False
    cell_content = []
    
    for line in lines:
        if "EXECUTE\n" in line:
            inside_cell = True
            cell_content = []  # Start a new cell
        elif "STDOUT/STDERR" in line:
            if inside_cell:
                cells.append("".join(cell_content).strip())
                inside_cell = False
        elif inside_cell:
            cell_content.append(line)
    
    return cells

def create_notebook_from_cells(cells, output_file):
    """
    Create a Jupyter notebook from a list of cell contents.
    """
    notebook = new_notebook()
    notebook.cells = [new_code_cell(cell) for cell in cells]
    
    with open(output_file, 'w') as f:
        nbformat.write(notebook, f)

def txt_to_ipynb_in_place(input_dir):
    """
    Convert all .txt files in the input directory and its subdirectories to .ipynb files.
    The generated notebooks are saved in the same folder as their corresponding .txt files.
    """
    for root, _, files in os.walk(input_dir):
        for file_name in files:
            if file_name.endswith('.txt'):
                input_file = os.path.join(root, file_name)
                output_file = os.path.join(root, file_name.replace('.txt', '.ipynb'))
                
                print(f"Processing {input_file}...")
                cells = extract_cells_from_txt(input_file)
                create_notebook_from_cells(cells, output_file)
                print(f"Created {output_file}")

if __name__ == "__main__":
    input_dir = "./"  # Replace with your root input directory
    
    txt_to_ipynb_in_place(input_dir)


Processing ./GPT4o\01\ipynb.txt...
Created ./GPT4o\01\ipynb.ipynb
